&nbsp;
&nbsp;

# Welcome to Feature Factory for Rossman Store Sales

Feature factory is an online infrastructure that allows one to quickly prototype and test features for different machine learning problems. 

Before beginning to use Feature Factory, we highly recommend that you familiarize yourself with what IPython Notebook. IPython Notebook is an interactive python kernel that allows you to run code in different cells. Variables created by the code live in the IPython Notebook python kernel and can be accessed at any time, by any cell. More information can be found at http://ipython.org/notebook.html

# Creating your own IPython Notebook

To get started with Feature Factory, please clone the Template notebook. To do this, click "File"->"Make a Copy". This should spawn a new tab within your browser with the copied notebook. Rename the notebook to your liking and make all edits on that notebook.

&nbsp;
&nbsp;


# Rossman Store Sales Machine Learning Competition

## Problem Statement

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

In this competition, you are challenged to identify and derive or generate the features which would help the most in making the prediction of 6 weeks of daily sales for the 1,115 stores located across Germany.

## Data

The dataset is in a relational format, split among mutliple files. When using **commands.get_sample_dataset()** to retrieve the dataset, the files are provided as a list of *pandas.DataFrame* objects.

The following step-by-step example shows this in detail.

### Sales Data

The sales table contains information about the sales which each store made each day within the analyzed period along with some information about the store circumstances on that date.

Note that each (Store, Date) combination is unique within the table.

| Data Fields | Definition |
|-------------|------------|
|Store        |a unique Id for each store corresdponding to *store* field in the *Stores* table|
|DayOfWeek    |Integer indicating the day of the week|
|Date         |Date when the sales took place|
|Sales        |the turnover for any given day (this is what needs to be predicted)|
|Customers    |the number of customers on a given day|
|Open         |an indicator for whether the store was open: 0 = closed, 1 = open|
|Promo        |indicates whether a store is running a promo on that day: 0 = store is running a promo, 1 = store is not running any promo|
|StateHoliday |indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None|
|SchoolHoliday|indicates if the (Store, Date) was affected by the closure of public schools|


### Stores Data

The stores table contains general information about each store, including store type, competition information and regular promotions.

|       Data Fields       | Definition |
|-------------------------|------------|
|Store                    |a unique Id for each store|
|StoreType                |differentiates between 4 different store models: a, b, c, d|
|Assortment               |describes an assortment level: a = basic, b = extra, c = extended|
|CompetitionDistance      |distance in meters to the nearest competitor store|
|CompetitionOpenSinceMonth|gives the approximate month of the time the nearest competitor was opened|
|CompetitionOpenSinceYear |gives the approximate year of the time the nearest competitor was opened|
|Promo2                   |Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating|
|Promo2SinceWeek          |describes the calendar week when the store started participating in Promo2|
|Promo2SinceYear          |describes the year when the store started participating in Promo2|
|PromoInterval            |describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store|

&nbsp;
&nbsp;

## Step-by-Step Example

Step 1: Import the feature factory infrastructure

In [183]:
from problems.rossmann import commands

&nbsp;
&nbsp;

Step 2: Create a username/password or login into an existing account. If you create an account and it is successful, you don't need to login - you are logged in automatically. 

In [2]:
commands.create_user('a_user', 'a_password')

username already exists


In [185]:
commands.login('stair_2', '')

user successfully logged in


&nbsp;
&nbsp;

Step 3: To ensure that this notebook is mapped to your username, it is required that you execute the command below. 

In [186]:
commands.add_notebook('Rossmann_Store_Sales_Stair_2')

Notebook already registered


&nbsp;
&nbsp;

Step 4: Get a sample dataset. This will allow you to test your feature before running it on the full data in the server. Remember that the dataset as a list of [Pandas DataFrames](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [187]:
dataset = commands.get_sample_dataset()
# dataset[0] <- this refers to the sales data
# dataset[1] <- this refers to the stores data


In [132]:
dataset[0][:5]

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2,2015-01-08,55,0,0,0,0
1,1,5,2013-01-12,49,1,0,0,1
2,1,6,2015-01-01,7338,1,1,0,0
3,1,2,2015-03-30,7388,1,0,0,0
4,1,1,2013-05-12,3385,1,0,0,0


In [18]:
dataset[1][:5]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


&nbsp;
&nbsp;

Step 5: Define your feature extraction function.

The name you give to the function is the name which will be used later on to register your feature extaction function and the score which it obtains.

Your function should simply take in the dataset list as a parameter and output a N x M numpy matrix or pandas dataframe where N is number of store-date pairings and M is the number of features which will be used for the prediction.
Bear in mind that sorting is important and that, in order to properly evaluate your function score, extracted features should preserve the order which the store-date pairings have in the sales dataset.

Also note that, even though the system allows you to do so, any feature extraction function which makes use of the outcome column will be disqualified.

**WARNING:** Your functions have to be self contained!

This means that you can use helper functions or import external modules but that any import or variable definition needs to be made within the functions which use them.

Cross validation is (intentionally) run in a separated process in order to make sure that this scope pattern is preserved, and will fail if the function uses anything defined somewhere else in the notebook.

You might be wondering why we require this. The reason is that the code of your function might be executed and further evaluated in different environments where the variables and modules defined in your notebook will not be available.

In [188]:
def RSS_SM01_Stair_2(dataset):

    import pandas as pd
    import warnings
    warnings.filterwarnings('ignore')

    data = dataset[0].merge(dataset[1], on='Store', sort=False).fillna(0)

    data['Date'] = pd.to_datetime(data['Date'])
    data['month'] = data['Date'].dt.month

    df = data[['CompetitionOpenSinceYear','CompetitionOpenSinceMonth']]
    df.columns = ['year', 'month']
    df['day'] = 1
    df.month = df.month.map(lambda x: x if x>0 else 1)
    df.year = df.year.map(lambda x: x if x>0 else 2016)

    df['year'] = df.year.map(lambda x: '%04d' % x)
    df['month'] = df.month.map(lambda x: '%02d' % x)
    df['day'] = df.day.map(lambda x: '%02d' % x)
    df = df.astype(str)
    data['CompetitionOpenSinceDate'] = pd.to_datetime(df.year + '-' + df.month + '-' + df.day, \
                                                      format="%Y%m%d",errors='ignore')
    data['CompetitionDuration'] = (data['Date'] - pd.to_datetime(data['CompetitionOpenSinceDate']))\
                                    .astype('timedelta64[D]')
    data['CompetitionDuration'] = data['CompetitionDuration'].map(lambda x: x if x>0 else 0)

    data['week'] = data['Date'].dt.weekofyear
    data['year'] = data['Date'].dt.year
    data['Promo2SinceYear'] = data.Promo2SinceYear.map(lambda x: x if x>0 else 2016)
    data['Promo2SinceWeek'] = data.Promo2SinceWeek.map(lambda x: x if x>0 else 53)

    data['PromoDuration'] = 53*data['year'] - 53*data['Promo2SinceYear']\
                            - data['Promo2SinceWeek'] + data['week']
    data['PromoDuration'] = data.PromoDuration.map(lambda x: x if x>0 else 0)

    data['st'] = data['StoreType'].astype("category")
    data['st'].cat.categories = range(4)
    data['assor'] = data['Assortment'].astype("category")
    data['assor'].cat.categories = range(3)
    data['pi'] = data['PromoInterval'].astype("category")
    data['pi'].cat.categories = range(4)
    data['sh'] = data['StateHoliday'].astype("category")
    data['sh'].cat.categories = range(4)


    data = data.drop(['Store', 'Date', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear' ,\
                     'CompetitionOpenSinceDate', 'Promo2SinceWeek', 'Promo2SinceYear',\
                      'week', 'year','Assortment', 'StoreType','PromoInterval',\
                      'StateHoliday'], axis=1)

    #from sklearn.decomposition import PCA
    #pca = PCA(n_components=2)
    #X=pca.fit_transform(data.as_matrix())
    return data


In [174]:
features = RSS_SM01_Stair_2(dataset)
features.head()

,DayOfWeek,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,Promo2,month,CompetitionDuration,PromoDuration,st,assor,pi,sh
0,2,55,0,0,0,1270.0,0,1,2320.0,0.0,2,0,0,0
1,5,49,1,0,1,1270.0,0,1,1594.0,0.0,2,0,0,0
2,6,7338,1,1,0,1270.0,0,1,2313.0,0.0,2,0,0,0
3,2,7388,1,0,0,1270.0,0,3,2401.0,0.0,2,0,0,0
4,1,3385,1,0,0,1270.0,0,5,1714.0,0.0,2,0,0,0


&nbsp;
&nbsp;

Step 6: Evaluate the score of your feature extraction function before submitting it.

You can make use of the cross_validate command as many times a needed in order to have a preview of what the score of your function will be.

In [189]:
commands.cross_validate(RSS_SM01_Stair_2)

Obtaining dataset
Extracting features


OSError: [Errno 12] Cannot allocate memory

&nbsp;
&nbsp;

Step 7: Register your function in the system

Once you are satisfied with the results, you can call the add_feature command passing your function as an argument.
This will cross_validate the function again and store your code and your score for future analysis.

Again, remember that your function code must be self contained and import or define everything it needs to be run successfully.

In [182]:
commands.add_feature(RSS_SM01_Stair_2)

Obtaining dataset
Extracting features


OSError: [Errno 12] Cannot allocate memory

&nbsp;
&nbsp;

Step 8: (Optional) Modify and update your function code.

If you discover that your function can be improved you can add it again into the system as many times as required with the same function name.

However, for improved clarity, we recommend you to use this option only to fix problems or make small improvements within a similar approach.

So, in case you want start a different feature extraction strategy, we strongly recommend you to register it with a new name.

In [9]:
def imports():
    """Make all the imports within our functions."""
    global pd
    import pandas as pd

def example_feature(dataset):
    """Return a dataset containing only some of the features."""
    imports()

    customers = dataset[0][['Customers']]
    promo_2 = dataset[0].merge(dataset[1], on='Store')['Promo2'].fillna(0)
    return pd.concat([customers, promo_2], axis=1)

commands.add_feature(example_feature)

Obtaining dataset
Extracting features
Cross validating
Your feature example_feature scored 0.8222449896321632
Feature example_feature successfully registered
